In [1]:
from azureml.core import Workspace, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core.environment import Environment

import pkg_resources
root_packages=['sklearn']

for m in pkg_resources.working_set:
    if m.project_name.lower() in root_packages:
        print(f"{m.project_name}=={m.version}")
import sys
!{sys.executable} -m pip install sklearn

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name='udacity-project')
project_env=Environment.from_conda_specification(name='project_env',file_path='./conda_dependencies.yml')

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

sklearn==0.0
Workspace name: quick-starts-ws-152708
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-152708


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Creating.......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive import HyperDriveRun
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C':uniform(0,1),'--max_iter':choice(1,1000)})


# Specify a Policy
policy = BanditPolicy(evaluation_interval=2,slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")
    
# Create a SKLearn estimator for use with train.py
#est = SKLearn(source_directory, *, compute_target=aml_compute, vm_size=None, vm_priority=None, entry_script=None)

# Running ScriptRun instead of estimator because estimator is depracated
src = ScriptRunConfig(source_directory='.',script='train.py',compute_target=aml_compute,environment=project_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

hd_config= HyperDriveConfig(run_config=src,hyperparameter_sampling=ps,policy=policy,primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=200)

#I made hyperdrive config runs>Logistic Regression runs

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.


hyperdrive_run = exp.submit(hd_config)

RunDetails(hyperdrive_run).show()

#Uncomment when Run is working properly
#run.wait_for_completion(show_output=True)




_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
#This section is only required when using HyperDriveStep

#Create the data store

#datastore = ws.get_default_datastore()

#Creating data folder

#data_folder = ds.as_named_input(name='bank_marketing')

#metrics_output_name = 'metrics_output'
#metrics_data = PipelineData(name='metrics_data',
#                            datastore=datastore,
#                           pipeline_output_name=metrics_output_name,
#                          training_output=TrainingOutput("Metrics"))

#model_output_name = 'model_output'
#saved_model = PipelineData(name='saved_model',
 #                           datastore=datastore,
  #                          pipeline_output_name=model_output_name,
   #                         training_output=TrainingOutput("Model",
    #                                                       model_file="outputs/model/saved_model.pb"))

#hd_step_name='hd_step_01'
#hd_step = HyperDriveStep(
 #   name=hd_step_name,
  #  hyperdrive_config=hd_config,
   # inputs=[data_folder],
    #outputs=[metrics_data, saved_model])

In [19]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Max iterations:',parameter_values[2])

joblib.dump(hyperdrive_run.get_hyperparameters(),'best_run.pkl')

print(parameter_values)

Best Run Id:  HD_e73b8b09-463c-4407-ac23-8f1c2f0e9b60_140

 Accuracy: 0.9179059180576631

 Max iterations: --max_iter
['--C', '0.3727289031169606', '--max_iter', '1000']


In [ ]:
#This section is only required when using HyperDriveStep
#conda_dep = CondaDependencies()
#conda_dep.add_pip_package("azureml-sdk")

#rcfg = RunConfiguration(conda_dependencies=conda_dep)

#register_model_step = PythonScriptStep(script_name='train.py',
 #                                      name='hd_step_01',
  #                                     inputs=[saved_model],
   #                                    compute_target=amlcompute_cluster_name,
    #                                   arguments=["--saved-model", saved_model],
     #                                  allow_reuse=True,
      #                                 runconfig=rcfg)

#register_model_step.run_after(hd_step)

#pipeline = Pipeline(workspace=ws, steps=[hd_step, register_model_step])
#pipeline_run = exp.submit(pipeline)

#RunDetails(pipeline_run).show()

#Waiting for the pipeline to run
#pipeline_run.wait_for_completion()

In [20]:
#Getting the metrics. This section is only used when using HyperDriveStep

#output_metrics= pipeline_run.get_pipeline_output(metrics_output_name)
#downloaded_num_file=output_metrics.download('.',show_progress=True)

#import pandas as pd
#import json

#with open(output_metrics._path_on_datastore) as i:
 #   output_metrics_results=i.read()
    
#output_metrics_decerialized=json.loads(output_metrics_results)
#output_metrics_df=pd.DataFrame(output_metrics_decerialized)

#output_metrics_df

In [21]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

auto_ml_url_path ='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

auto_ml_ds = TabularDatasetFactory.from_delimited_files(path=auto_ml_url_path)

In [22]:
from train import clean_data

# Use the clean_data function to clean your data.
auto_ml_x, auto_ml_y = clean_data(auto_ml_ds)
auto_ml_x['y']=auto_ml_y

auto_ml_ds_cleaned=auto_ml_x

In [23]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=auto_ml_ds,
    label_column_name="y",
    n_cross_validations=8)

In [24]:
# Submit your automl run

auto_ml_experiment=Experiment(ws,"auto_ml_experiment")
auto_ml_run=auto_ml_experiment.submit(config=automl_config,show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
auto_ml_experiment,AutoML_3ae3948f-5b93-4470-a951-b1b74914037e,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias toward

In [30]:
# Retrieve and save your best automl model.

best_auto_ml_model,fitted_auto_ml_model=auto_ml_run.get_output()

model_name = best_auto_ml_model.properties['model_name']
description = 'AutoML bank marketing example'
tags = None

#Register only when AutoMl is done
#model = run.register_model(model_name = model_name, description = description, tags = tags)
